<a href="https://colab.research.google.com/github/sherryfengshopify/ai-gym/blob/main/ReAct_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai

In [ ]:
!pip install --upgrade openai httpx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.9/389.9 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.0
    Uninstalling httpx-0.28.0:
      Successfully uninstalled httpx-0.28.0
  Attempting uninstall: openai
    Found existing installation: openai 1.54.5
    Uninstalling openai-1.54.5:
      Successfully uninstalled openai-1.54.5


In [ ]:
# @title Import and setup
from google.cloud import bigquery
from google.colab import data_table
import bigframes.pandas as bpd
import pandas as pd
from openai import OpenAI
import json
import yaml

# project = 'sdp-prd-build-ecosystem' # Project ID inserted based on the query results selected to explore
# location = 'us-central1' # Location inserted based on the query results selected to explore

llm = OpenAI(
    base_url="https://openai-proxy.shopify.ai/v1",
    api_key="shopify-eyJpZCI6IjcyNTA1MGNjN2JmOThiOWEwMGE5ODAyMGZiM2U1NWM0IiwibW9kZSI6InBlcnNvbmFsIiwiZW1haWwiOiJzaGVycnkuZmVuZ0BzaG9waWZ5LmNvbSIsImV4cGlyeSI6MTczMzU4ODE0Nn0=-GfKu6/4M7W+R7DXi03LvG+P2zjFRp5Q6NPrU/+oh7Fw="
)

# client = bigquery.Client(project=project, location='US')
# data_table.enable_dataframe_formatter()

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
import openai
import re
import httpx
import os

In [ ]:
# client = OpenAI()

MODEL = 'gpt-4o-mini'
prompt = 'Write something short but funny.'

chat_completion = llm.chat.completions.create(
    model=MODEL,
    messages=[
        {
            'role': 'user',
            'content': prompt,
        }
    ]
)

In [ ]:
chat_completion.choices[0].message.content

'Why did the scarecrow win an award? Because he was outstanding in his field!'

In [ ]:
# @title Creating the agent class

class Agent:
  def __init__(self, system=''):
    self.system = system
    self.messages = []

    if self.system:
      self.messages.append({'role': 'system', 'content': system})

  def __call__(self, prompt):
    self.messages.append({'role': 'user', 'content': prompt})
    result = self.execute()
    self.messages.append({'role': 'assistant', 'content': result})
    return result

  def execute(self, model='gpt-4o-mini', temperature=0):
    completion = llm.chat.completions.create(
        model=model,
        temperature=temperature,
        messages=self.messages
    )
    return completion.choices[0].message.content

In [ ]:
# @title Creating the ReAct prompt
prompt = '''
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

get_cost:
e.g. get_cost: book
returns the cost of a book

wikipedia:
e.g. wikipedia: LangChain
Returns a summary from searching Wikipedia

Always look things up on Wikipedia if you have the opportunity to do so.

Example session #1:

Question: How much does a pen cost?
Thought: I should look the pen cost using get_cost
Action: get_cost: pen
PAUSE

You will be called again with this:

Observation: A pen costs $5


Example session #2:

Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:

Observation: France is a country. The capital is Paris.

You then output:

Answer: The capital of France is Paris.

'''.strip()

In [ ]:
# @title Creating the tools

# 1. the calculate() function takes in a string, evaluates that string, and returns the result
def calculate(what):
  return eval(what)

# 2. the get_cost() function returns the cost for a pen, a book, and a stapler
def get_cost(thing):
  if thing in 'pen':
    return('A pen costs $5')
  elif thing in 'book':
    return('A book costs $20')
  elif thing in 'stapler':
    return('A stapler costs $10')
  else:
    return('A random thing for writing costs $12.')

#3. the wikipedia() function uses the Wikipedia API to search for a specific query on Wikipedia
def wikipedia(q):
  response = httpx.get('https://en.wikipedia.org/w/api.php', params={
      'action': 'query',
      'list': 'search',
      'srsearch': q,
      'format': 'json',
  })
  results = response.json().get('query').get('search', [])

  if not results:
    return None
  return results[0]['snippet']

In [ ]:
wikipedia('langchain')

'<span class="searchmatch">LangChain</span> is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration'

In [ ]:
# dictionary that maps the function names to the function themselves
known_actions = {
    'calculate': calculate,
    'get_cost': get_cost,
    'wikipedia': wikipedia
}

In [ ]:
# @title Testing the agent
my_agent = Agent(prompt)

In [ ]:
# calling the agent with this initial question
result = my_agent('How much does a pen cost?')
print(result)

Thought: I should look up the cost of a pen using the get_cost action.  
Action: get_cost: pen  
PAUSE


In [ ]:
# creating the next prompt that will be used as an observation and passed to the language model
next_prompt = f"Observation: {get_cost('pen')}"

In [ ]:
# calling the agent with that next prompt
my_agent(next_prompt)

'Answer: A pen costs $5.'

In [ ]:
my_agent.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_cost:\ne.g. get_cost: book\nreturns the cost of a book\n\nwikipedia:\ne.g. wikipedia: LangChain\nReturns a summary from searching Wikipedia\n\nAlways look things up on Wikipedia if you have the opportunity to do so.\n\nExample session #1:\n\nQuestion: How much does a pen cost?\nThought: I should look the pen cost using get_cost\nAction: get_cost: pen\nPAUSE\n\nYou will be called again with this:\n\nObservation: A pen costs $5\n\n\nExample session #2:\n\nQues

In [ ]:
abot = Agent(prompt)

In [ ]:
question = '''I want to buy a pen and a book.
How much do they cost in total?'''

abot(question)

'Thought: I need to find the cost of both a pen and a book to calculate the total cost. I will first get the cost of each item using the get_cost action. \nAction: get_cost: pen\nPAUSE'

In [ ]:
# executing the action and creating the next prompt.
next_prompt = f'Observation: {get_cost("pen")}'
print(next_prompt)

Observation: A pen costs $5


In [ ]:
abot(next_prompt)

'Thought: Now that I have the cost of the pen, I need to find the cost of the book to calculate the total. \nAction: get_cost: book\nPAUSE'

In [ ]:
next_prompt = f'Observation: {get_cost("book")}'
print(next_prompt)

Observation: A book costs $20


In [ ]:
abot(next_prompt)

'Thought: I now have the costs of both the pen and the book. The pen costs $5 and the book costs $20. I will calculate the total cost by adding these two amounts together. \nAction: calculate: 5 + 20\nPAUSE'

In [ ]:
abot = Agent(prompt)

In [ ]:
query = '2024 United Kingdom elections'
abot(query)

'Thought: I should look up information about the 2024 United Kingdom elections on Wikipedia to get the most accurate and detailed information.  \nAction: wikipedia: 2024 United Kingdom elections  \nPAUSE'

In [ ]:
# calling the wikipedia() function and creating the next prompt
next_prompt = f'Observation: {wikipedia(query)}'
print(next_prompt)

Observation: The <span class="searchmatch">2024</span> <span class="searchmatch">United</span> <span class="searchmatch">Kingdom</span> general <span class="searchmatch">election</span> was held on Thursday, 4 July <span class="searchmatch">2024</span>, to elect 650 members of Parliament to the House of Commons, the lower house


In [ ]:
# calling the agent with the new prompt, which in turn is gving the final answer
abot(next_prompt)

'Answer: The 2024 United Kingdom general election was held on Thursday, 4 July 2024, to elect 650 members of Parliament to the House of Commons, the lower house.'

In [ ]:
abot.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_cost:\ne.g. get_cost: book\nreturns the cost of a book\n\nwikipedia:\ne.g. wikipedia: LangChain\nReturns a summary from searching Wikipedia\n\nAlways look things up on Wikipedia if you have the opportunity to do so.\n\nExample session #1:\n\nQuestion: How much does a pen cost?\nThought: I should look the pen cost using get_cost\nAction: get_cost: pen\nPAUSE\n\nYou will be called again with this:\n\nObservation: A pen costs $5\n\n\nExample session #2:\n\nQues